In [1]:
# Install latest version from GitHub
# !pip install -q -U git+https://github.com/jdvelasq/techminer

In [2]:
import json

import matplotlib.pyplot as plt
import pandas as pd

from techminer import (
    DataFrame,
    Plot,
    Thesaurus,
    extract_country,
    heatmap,
    text_clustering,
    text_nesting,
)

In [3]:
filepath = (
    "https://raw.githubusercontent.com/jdvelasq/techminer/master/data/"
    + "ieee-latin-america.csv"
)

filepath = "../../data/papers/ieee-latin-america.csv"

In [4]:
df = pd.read_csv(filepath)
df = df.applymap(lambda x: None if pd.isna(x) is True else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Authors                    3553 non-null   object 
 1   Author(s) ID               3553 non-null   object 
 2   Title                      3553 non-null   object 
 3   Year                       3553 non-null   int64  
 4   Source title               3553 non-null   object 
 5   Volume                     3553 non-null   int64  
 6   Issue                      3553 non-null   int64  
 7   Art. No.                   3067 non-null   float64
 8   Page start                 3525 non-null   float64
 9   Page end                   3498 non-null   float64
 10  Page count                 11 non-null     float64
 11  Cited by                   2452 non-null   float64
 12  DOI                        3538 non-null   object 
 13  Link                       3553 non-null   objec

In [5]:
#
# Number of records
#
len(df)

3553

In [6]:
#
# Keywords merging
#
df = DataFrame(df).keywords_fusion()
df.Keywords.head()

0    3D modeling;Bottleneck identifications;Computa...
1    Combinated Models;Combinatorial Network of Dyn...
2    Accident prevention strategies;Critical system...
3    Distribution system;Distribution systems;Elect...
4    Assistive Technology;Contact surface;Digital s...
Name: Keywords, dtype: object

In [ ]:
#
# Number of records without Keywords
#
len(df[df.Keywords.map(lambda x: x is None)])

In [ ]:
#
# Titles of documents without keywords
#
df[df.Keywords.map(lambda x: x is None)]['Title']

In [ ]:
#
# Delete of records without keywords
#
df = df[df.Keywords.map(lambda x: x is not None)]

In [ ]:
#
# Number of selected records
#
len(df)

In [ ]:
#
# Text clustering of keywords.
#   Keyword strings with the same number of words
#
thesaurus = text_clustering(df.Keywords, sep=";", transformer=lambda x: x.lower())
with open("thesaurus-text-clustering-raw.json", "w") as f:
    f.write(thesaurus.__repr__())

In [ ]:
x = pd.Series(['abc', 'ab', 'a']).tolist()
sorted(x, key=len)

In [ ]:
#
#   Loads the new dictionary
#
with open("thesaurus-text-clustering-cleaned.json", "r") as f:
    dictionary = json.loads(f.read())
    
#
#   Loads the new thesaurus
#
thesaurus = Thesaurus(dictionary, ignore_case=False, full_match=True, use_re=False)

#
#   Clean the Keywords
#
thesaurus.compile()
df['Keywords'] = df.Keywords.map(lambda x: thesaurus.apply_as_dict(x, sep=";"))
# thesaurus_dict = thesaurus.to_dict()
# df['Keywords'] = df.Keywords.map(lambda x: ';'.join([thesaurus_dict[w] if w in thesaurus_dict.keys() else w for w in x.split(';')]))


#
#   Remove extra blanks between keywords if exists
#
df["Keywords"] = df.Keywords.map(
    lambda x: ";".join(set([w.strip() for w in x.split(";")]))
)

#
#   Replace empty strings by None
#
df["Keywords"] = df.Keywords.map(lambda x: x if x != "" else None)

#
#   Number of unique of strings
#
len(
    set([w.strip() for x in df.Keywords if x is not None for w in x.split(";")])
)

In [ ]:
#
# Text nesting
#
thesaurus = text_nesting(df.Keywords, sep=';', max_distance=1, transformer=lambda x: x.lower())

#
# Creates a thesaurus with candidate substrings as a thesaurus
#
with open('thesaurus-text-nesting-raw.json', 'w') as f:
    f.write(tn.__repr__())

In [ ]:
#
#   Loads the new dictionary
#
with open("thesaurus-text-nesting-cleaned.json", "r") as f:
    dictionary = json.loads(f.read())
    
#
#   Loads the new thesaurus
#
thesaurus = Thesaurus(dictionary, ignore_case=False, full_match=True, use_re=False)

#
# Apply the thesaurus
#
df["Keywords"] = df.Keywords.map(
    lambda x: thesaurus.apply(x, sep=";")
)

#
# Remove extra blanks between keywords
#
df["Keywords"] = df.Keywords.map(
    lambda x: ";".join(set([w.strip() for w in x.split(";")]))
)

#
# Replace empty strings by None
#
df["Keywords"] = df.Keywords.map(
    lambda x: x if x != "" else None
)

#
# Number of unique keywords
#
len(
    set(
        [w.strip() for x in df.Keywords if x is not None for w in x.split(";")]
    )
)

In [ ]:
#
# A column for identify each record is added to the dataframe.
#


df = DataFrame(df).generate_ID()

In [ ]:
#
# Number of terms
#
df.count_report()

In [ ]:
#
# Author desambiguation
#
df = DataFrame(df).disambiguate_authors()

In [ ]:
#
# Top 10 most frequent authors
#
sorted(df.documents_by_term('Authors').head(10).Authors)

In [ ]:
#
# Top 10 most cited authors
#
sorted(df.citations_by_term('Authors').head(10).Authors)

In [ ]:
#
# Top 10 most frequent keywords
#
df.documents_by_term("Keywords", sep=";").head(10)

In [ ]:
#
# Top 10 most cited keywords
#
df.citations_by_term("Keywords", sep=";").head(10)

In [ ]:
#
# Documents by year
#


Plot(df.documents_by_year()).bar(cmap=plt.cm.Blues)

In [ ]:
#
# Cumulative number of documents by year
#
Plot(df.documents_by_year(cumulative=True)).barh()

In [ ]:
#
# Worldmap with the number of documents by country
#
df["Country"] = df["Affiliations"].map(lambda x: extract_country(x, sep=";"))
plt.figure(figsize=(12,5))
Plot(df.documents_by_term("Country", sep=";")).worldmap()
plt.show()